In [1]:
library(Signac)
library(Seurat)
library(GenomeInfoDb)
library(BSgenome.Hsapiens.UCSC.hg38)
library(EnsDb.Hsapiens.v86)
library(BSgenome.Mmusculus.UCSC.mm10)
library(EnsDb.Mmusculus.v79)
library(ggplot2)
library(patchwork)
library(Matrix)
library(GenomicRanges)
library(tidyverse)
set.seed(1234)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following obj

In [2]:
#
LoadData <- function(data_path=NULL, assay='ATAC'){
    if (assay=='ATAC'){
        atac_assay=readMM(paste0(data_path,'/matrix.mtx'))
        barcodes=readLines(paste0(data_path,'/barcodes.tsv'))
        peaks=readLines(paste0(data_path,'/peak.bed'))
        row.names(atac_assay)=peaks
        colnames(atac_assay)=barcodes
        #
        return (atac_assay)
    }
    if (assay=='RNA'){
        rna_assay=readMM(paste0(data_path,'/matrix.mtx'))
        barcodes=readLines(paste0(data_path,'/barcodes.tsv'))
        features=readLines(paste0(data_path,'/genes.tsv'))
        row.names(rna_assay)=features
        colnames(rna_assay)=barcodes
        #
        return (rna_assay)
    }
    if (assay=='GeneScore'){
        rna_assay=readMM(paste0(data_path,'/matrix.mtx'))
        barcodes=readLines(paste0(data_path,'/barcodes.tsv'))
        features=readLines(paste0(data_path,'/genes.tsv'))
        row.names(rna_assay)=features
        colnames(rna_assay)=barcodes
        #
        return (rna_assay)
    }
}
#
Processing_peaks <- function(peak_file=NULL, sep = c(":", "-"), remove_genome='mm10'){
    # loading
    peaks <- readLines(peak_file)
    
    # processing
    if (remove_genome=='mm10'){
        peaks=peaks[!grepl(paste0(c('mm10'), collapse = "|"), peaks)]
        peaks=gsub('GRCh38_','',peaks)
    }
    if (remove_genome=='hg19'){
        peaks=peaks[!grepl(paste0(c('GRCh38'), collapse = "|"), peaks)]
        peaks=gsub('mm10_','',peaks)
    }
    
    # convert to genomic ranges
    gr.peaks <- StringToGRanges(regions = peaks, sep = sep)
    # standard peaks
    gr.peaks <- Standard_peaks(gr.peaks)
    return (gr.peaks)
}
#
Processing_counts <- function(counts=NULL, remove_genome='mm10'){
    if (remove_genome=='mm10'){
        peaks=row.names(counts)
        peaks=peaks[!grepl(paste0(c('mm10'), collapse = "|"), peaks)]
        counts=counts[peaks,]
        #
        peaks=row.names(counts)
        peaks=gsub('hg19_','',peaks)
        row.names(counts)=peaks
        return (counts)
    }
    ###
    if (remove_genome=='hg19'){
        peaks=row.names(counts)
        peaks=peaks[!grepl(paste0(c('hg19'), collapse = "|"), peaks)]
        counts=counts[peaks,]
        #
        peaks=row.names(counts)
        peaks=gsub('mm10_','',peaks)
        row.names(counts)=peaks
        return (counts)
    }
}
Standard_peaks <- function (rowRanges=rowRanges, annotation='hg38'){
    if (annotation=='hg19'){main.chroms <- standardChromosomes(BSgenome.Hsapiens.UCSC.hg19)}
    if (annotation=='hg38'){main.chroms <- standardChromosomes(BSgenome.Hsapiens.UCSC.hg38)}
    if (annotation=='mm10'){main.chroms <- standardChromosomes(BSgenome.Mmusculus.UCSC.mm10)}
    keep.peaks <- which(as.character(seqnames(rowRanges)) %in% main.chroms)
    #keep.peaks <- which(as.character(seqnames(rowRanges)) %in% c('chr1'))
    rowRanges <- rowRanges[keep.peaks]
    
    #peaks=as.data.frame(rowRanges)[,c(1,2,3)]
    #peaks=peaks[keep.peaks,]
    #rowRanges <- makeGRangesFromDataFrame(peaks)
    
    return (rowRanges)
}
Create_scRNA_object <- function(data.dir=NULL, rna.assay=NULL){
    #
    if (is.null(data.dir)){
        if (is.null(rna.assay)){stop('Please provide the RNA expression matrix')}
    }
    else{
        rna.assay=Read10X(data.dir = data.dir, gene.column = 1)
        #rna.assay=Read10X(data.dir = data.dir, gene.column = 2)
    }
    #
    proj <- CreateSeuratObject(counts = rna.assay)
    #
    return(proj)
}
Create_scATAC_object <- function(counts=NULL,fragment_file=NULL){
    #
    atac.assay <- CreateChromatinAssay(counts, fragments = fragment_file)
    atac.obj <- CreateSeuratObject(atac.assay, assay = "ATAC")
    #
    return (atac.obj)
}
# Do annotation for seurat object
annotate_ATAC_obj <- function(proj=proj, annotation='hg38'){
    #
    if (annotation=='hg38'){
        hg38 = "/jdfssz1/ST_SUPERCELLS/PUB/scRNA/pipeline/v3.1.5/database/Human/genes.gtf"
        gtf <- rtracklayer::import(hg38)
        gene.coords <- gtf[gtf$type == 'gene']
        seqlevelsStyle(gene.coords) <- 'UCSC'
        gene.coords <- keepStandardChromosomes(gene.coords, pruning.mode = 'coarse')
        gene.coords$gene_biotype <- gene.coords$gene_type
    }
    #
    if (annotation=='mm10'){
        mm10 = "/jdfssz1/ST_SUPERCELLS/PUB/scRNA/pipeline/v3.1.5/database/Mouse/genes.gtf"
        gtf <- rtracklayer::import(mm10)
        gene.coords <- gtf[gtf$type == 'gene']
        seqlevelsStyle(gene.coords) <- 'UCSC'
        gene.coords <- keepStandardChromosomes(gene.coords, pruning.mode = 'coarse')
        gene.coords$gene_biotype <- gene.coords$gene_type
    }
    # add the gene information to the object
    Annotation(proj) <- gene.coords
    #
    return (proj)
}

# nFeature/nCounts per cell

## ATAC

### single sample

In [3]:
data_path='/hwfssz1/ST_SUPERCELLS/P21Z10200N0090/zhangzhongjin/celline/cellline_ISSAAC_processed/IL10/ATAC/downsampled/'
sample_names=c()
names=c('HT-scCAT-seq_EU11_1')
human=c('HEK293T')
mouse=c('NIH3T3')
counts_files_human=c(paste0(data_path, human))
counts_files_mouse=c(paste0(data_path, mouse))

In [4]:
for (i in 1:length(counts_files_human)){
    counts <- LoadData(data_path=counts_files_human)
    counts <- Processing_counts(counts, remove_genome='mm10')
    seurat.obj=Create_scATAC_object(counts=counts, fragment_file=NULL)
    meta=seurat.obj@meta.data[, c('nCount_ATAC', 'nFeature_ATAC')]
    colnames(meta)=c('nCounts', 'nFeatures')
    #
    output_dir='/hwfssz1/ST_SUPERCELLS/P21Z10200N0090/zhangzhongjin/celline/cellline_ISSAAC_processed/IL10/ATAC/downsampled/'
    output_path=paste0(output_dir, human)
    if (!dir.exists(output_path)){dir.create(output_path)}
    write.csv(meta, paste0(output_path,'/metrics.csv'), quote=FALSE)
}
for (i in 1:length(counts_files_mouse)){
    counts <- LoadData(data_path=counts_files_mouse)
    counts <- Processing_counts(counts, remove_genome='hg19')
    seurat.obj=Create_scATAC_object(counts=counts, fragment_file=NULL)
    meta=seurat.obj@meta.data[, c('nCount_ATAC', 'nFeature_ATAC')]
    colnames(meta)=c('nCounts', 'nFeatures')
    #
    output_dir='/hwfssz1/ST_SUPERCELLS/P21Z10200N0090/zhangzhongjin/celline/cellline_ISSAAC_processed/IL10/ATAC/downsampled/'
    output_path=paste0(output_dir, mouse)
    if (!dir.exists(output_path)){dir.create(output_path)}
    write.csv(meta, paste0(output_path,'/metrics.csv'), quote=FALSE)
}

### multi-samples

In [5]:
data_path='/jdfssz1/ST_SUPERCELLS/P21Z10200N0090/Automated/USER/zhangzhongjin/project/celline/sample/all_tech/'
sample_names=c('CAT_EU11_1', 'CAT_EU14', 'CAT_FL16_1')
names=c('HT-scCAT-seq_EU11_1', 'HT-scCAT-seq_EU14', 'HT-scCAT-seq_FL16_1')
human=c('HEK293T')
mouse=c('NIH3T3')
ATAC=c('ATAC')
#
counts_files_human=c()
counts_files_mouse=c()
for (i in 1:length(sample_names)){
    counts_files_human=c(counts_files_human, paste0(data_path, sample_names[i],"/", ATAC,"/", human))
    counts_files_mouse=c(counts_files_mouse, paste0(data_path, sample_names[i],"/", ATAC,"/", mouse))
}

In [6]:
names_human=c()
names_mouse=c()
for (i in 1:length(names)){
names_human[i]=c(paste0(names[i], '_', human))
names_mouse[i]=c(paste0(names[i], '_', mouse))
    }

In [7]:
for (i in 1:length(counts_files_human)){
    counts <- LoadData(data_path=counts_files_human[i])
    counts <- Processing_counts(counts, remove_genome='mm10')
    seurat.obj=Create_scATAC_object(counts=counts, fragment_file=NULL)
    meta=seurat.obj@meta.data[, c('nCount_ATAC', 'nFeature_ATAC')]
    colnames(meta)=c('nCounts', 'nFeatures')
    #
    output_dir='/jdfssz1/ST_SUPERCELLS/P21Z10200N0090/Automated/USER/zhangzhongjin/project/celline/sample/compare_data/ATAC/'
    output_path=paste0(output_dir, names_human[i])
    if (!dir.exists(output_path)){dir.create(output_path)}
    write.csv(meta, paste0(output_path,'/metrics.csv'), quote=FALSE)
}

In [8]:
for (i in 1:length(counts_files_mouse)){
    counts <- LoadData(data_path=counts_files_mouse[i])
    counts <- Processing_counts(counts, remove_genome='hg19')
    seurat.obj=Create_scATAC_object(counts=counts, fragment_file=NULL)
    meta=seurat.obj@meta.data[, c('nCount_ATAC', 'nFeature_ATAC')]
    colnames(meta)=c('nCounts', 'nFeatures')
    #
    output_dir='/jdfssz1/ST_SUPERCELLS/P21Z10200N0090/Automated/USER/zhangzhongjin/project/celline/sample/compare_data/ATAC/'
    output_path=paste0(output_dir, names_mouse[i])
    if (!dir.exists(output_path)){dir.create(output_path)}
    write.csv(meta, paste0(output_path,'/metrics.csv'), quote=FALSE)
}

## RNA

### single sample

In [9]:
data_path='/hwfssz1/ST_SUPERCELLS/P21Z10200N0090/zhangzhongjin/celline/cellline_ISSAAC_processed/ISSAAC/RNA/downsampled/'
sample_names=c()
names=c('HT-scCAT-seq_IL9')
human=c('HEK293T')
mouse=c('NIH3T3')
counts_files_human=c(paste0(data_path, human))
counts_files_mouse=c(paste0(data_path, mouse))
counts_files_human
counts_files_mouse

[1] "/hwfssz1/ST_SUPERCELLS/P21Z10200N0090/zhangzhongjin/celline/cellline_ISSAAC_processed/ISSAAC/RNA/downsampled/HEK293T"

[1] "/hwfssz1/ST_SUPERCELLS/P21Z10200N0090/zhangzhongjin/celline/cellline_ISSAAC_processed/ISSAAC/RNA/downsampled/NIH3T3"

In [10]:
# HEK293T
counts <- LoadData(data_path=counts_files_human, assay='RNA')
counts <- as(counts, 'CsparseMatrix')
seurat.obj=CreateSeuratObject(counts=counts)
meta=seurat.obj@meta.data[, c('nCount_RNA', 'nFeature_RNA')]
colnames(meta)=c('nCounts', 'nFeatures')
output_dir='/hwfssz1/ST_SUPERCELLS/P21Z10200N0090/zhangzhongjin/celline/cellline_ISSAAC_processed/ISSAAC/RNA/downsampled/HEK293T'
write.csv(meta, paste0(output_dir,'/metrics.csv'), quote=FALSE)
# NIH3T3
counts <- LoadData(data_path=counts_files_mouse, assay='RNA')
counts <- as(counts, 'CsparseMatrix')
seurat.obj=CreateSeuratObject(counts=counts)
meta=seurat.obj@meta.data[, c('nCount_RNA', 'nFeature_RNA')]
colnames(meta)=c('nCounts', 'nFeatures')
output_dir='/hwfssz1/ST_SUPERCELLS/P21Z10200N0090/zhangzhongjin/celline/cellline_ISSAAC_processed/ISSAAC/RNA/downsampled/NIH3T3'
write.csv(meta, paste0(output_dir,'/metrics.csv'), quote=FALSE)

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"


### multi-samples

In [11]:
data_path='/hwfssz1/ST_SUPERCELLS/P21Z10200N0090/zhangzhongjin/celline/sample/cortex/EL13/RNA/merged/'
sample_names=c('CAT_EU11_1', 'CAT_EU14', 'CAT_FL16_1')
names=c('HT-scCAT-seq_EU11_1', 'HT-scCAT-seq_EU14', 'HT-scCAT-seq_FL16_1')
human=c('HEK293T')
mouse=c('NIH3T3')
RNA=c('RNA')
#
counts_files_human=c()
counts_files_mouse=c()
for (i in 1:length(sample_names)){
    counts_files_human=c(counts_files_human, paste0(data_path, sample_names[i],"/", RNA,"/", human))
    counts_files_mouse=c(counts_files_mouse, paste0(data_path, sample_names[i],"/", RNA,"/", mouse))
}

In [12]:
for (i in 1:length(counts_files_human)){
    counts <- LoadData(data_path=counts_files_human[i], assay='RNA')
    counts <- as(counts, 'CsparseMatrix')
    seurat.obj=CreateSeuratObject(counts=counts)
    meta=seurat.obj@meta.data[, c('nCount_RNA', 'nFeature_RNA')]
    colnames(meta)=c('nCounts', 'nFeatures')
    #
    output_dir='/jdfssz1/ST_SUPERCELLS/P21Z10200N0090/Automated/USER/zhangzhongjin/project/celline/sample/compare_data/RNA/'
    output_path=paste0(output_dir, names_human[i])
    if (!dir.exists(output_path)){dir.create(output_path)}
    write.csv(meta, paste0(output_path,'/metrics.csv'), quote=FALSE)
}

In [13]:
for (i in 1:length(counts_files_mouse)){
    counts <- LoadData(data_path=counts_files_mouse[i], assay='RNA')
    counts <- as(counts, 'CsparseMatrix')
    seurat.obj=CreateSeuratObject(counts=counts)
    meta=seurat.obj@meta.data[, c('nCount_RNA', 'nFeature_RNA')]
    colnames(meta)=c('nCounts', 'nFeatures')
    #
    output_dir='/jdfssz1/ST_SUPERCELLS/P21Z10200N0090/Automated/USER/zhangzhongjin/project/celline/sample/compare_data/RNA/'
    output_path=paste0(output_dir, names_mouse[i])
    if (!dir.exists(output_path)){dir.create(output_path)}
    write.csv(meta, paste0(output_path,'/metrics.csv'), quote=FALSE)
}

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
